In [1]:
import sys
sys.path.insert(0,'../../backend')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset, generate_expert_traj

import webotsgym as wg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from webotsgym.utils import exponential_decay
from webotsgym.env.reward import step_pen_exp

class MyEval(wg.WbtReward):
    def __init__(self, env, config):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        target_distance = self.env.get_target_distance(False)
        if target_distance < 0.1:
            print("Calc_reward target bonus: ", 5000 + 5000 * (1 - abs(self.env.state.speed)/0.03))
            return 5000 + 5000 * (1 - abs(self.env.state.speed)/0.03)
        else:
            reward = 0
            reward += -10
            env.step_pen -= 10
            
            if len(self.env.distances) > 0 and len(self.env.history) > 0:
                gps_start_0 = self.env.history[0].gps_actual[0] - self.env.gps_target[0]
                gps_start_1 = self.env.history[0].gps_actual[1] - self.env.gps_target[1]
                
                start_dist = math.sqrt(gps_start_0**2 + gps_start_1**2)
                
                
                
                target_diff = self.env.distances[-1] - self.env.get_target_distance(False)
                
                gps_diff_0 = self.env.history[-1].gps_actual[0] - self.env.state.gps_actual[0]
                gps_diff_1 = self.env.history[-1].gps_actual[1] - self.env.state.gps_actual[1]
                
                #print("history: ", self.env.history[-1].gps_actual[0])
                #print("actual:  " , self.env.state.gps_actual[0])
                
                actual_diff = math.sqrt(gps_diff_0**2 + gps_diff_1**2)
                
                if actual_diff > 0:
                    diff_rew = abs(target_diff / start_dist * 5000) * ( target_diff  / actual_diff)
                else:
                    diff_rew = 0
                    
                                
                env.dist_rew.append(diff_rew)
                if sum(env.dist_rew) > 5000:
                    print("Dist_rew too big: ", sum(env.dist_rew))
                    
                env.t_diff.append(target_diff)
                if sum(env.t_diff) > start_dist:
                    print("Sum of t_diff to big: ", sum(env.t_diff))
                    print("start_dist: ",start_dist)
               
                    
                #if diff_rew < 0:
                #    diff_rew = diff_rew * 2
                
                #print("Target_diff: ", target_diff)
                #print("Actual_diff: ", actual_diff)
                #print("start_dist: ", start_dist)
                #print("diff_rew:   ", diff_rew)  
                    
  
                reward += diff_rew
                    
            if self.env.state.action_denied:
                reward += -10
        
            if self.env.state.touching:
                reward += -200
        return reward

    def check_done(self):
        if self.env.total_reward < -5000:
            print("reward boundary, reward: ", self.env.total_reward)
            print("              dist_rew : ", sum(env.dist_rew))
            env.dist_rew.clear()
            env.t_diff.clear()
            env.step_pen = 0
            return True
        
        if self.env.step_pen < -2000:
            print("reward boundary, reward: ", self.env.total_reward)
            print("              dist_rew : ", sum(env.dist_rew))
            env.dist_rew.clear()
            env.t_diff.clear()
            env.step_pen = 0
            return True
        
        if self.env.get_target_distance(False) < 0.1:
            print("target reached, reward: ", self.env.total_reward)
            print("              dist_rew : ", sum(env.dist_rew))
            env.dist_rew.clear()
            env.t_diff.clear()
            env.step_pen = 0
            return True
        
        if self.env.total_reward > 25000:
            print("Stop hacking!, reward: ", self.env.total_reward)
            print("              dist_rew : ", sum(env.dist_rew))
            env.dist_rew.clear()
            env.t_diff.clear()
            env.step_pen = 0
            return True
        
        return False

In [3]:
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.FAST
config.sim_step_every_x = 10
config.relative_action = True
config.direction_type = wg.config.DirectionType.STEERING

In [4]:
env = wg.WbtGym(train=True, 
                evaluate_class=MyEval,
                action_class = wg.WbtActContinuous(config=config, relative = True),
                config=config)

../../backend/webotsgym/env/webotenv.py:123: UserWarning: Relative property of action class is overwritten by config.relative_action. This might interfere with bounds argument for WbtActContinuous.
  warnings.warn("Relative property of action class is overwritten "


Accepting on Port:  10201


In [5]:
env.dist_rew = []
env.t_diff = []
env.step_pen = 0

In [6]:
model_name = "3x3_reach_target_v7_b"
model = PPO1("MlpPolicy", env, timesteps_per_actorbatch = 5000, tensorboard_log="./{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
while True:
    model.learn(total_timesteps=500000)
    model.save("./model_{}".format(model_name))


reward boundary, reward:  -5303.018350164373
              dist_rew :  -1573.0183501643728
reward boundary, reward:  -1131.8411488231866
              dist_rew :  1878.1588511768161
reward boundary, reward:  -5202.758038568446
              dist_rew :  -52.75803856844497
reward boundary, reward:  -5098.165453368743
              dist_rew :  1011.8345466312575
reward boundary, reward:  -5243.583650487784
              dist_rew :  -983.5836504877822
reward boundary, reward:  -5084.801317162799
              dist_rew :  -2944.801317162801
reward boundary, reward:  -2782.4448873668885
              dist_rew :  227.55511263311178
reward boundary, reward:  -5093.467736244185
              dist_rew :  -1473.467736244188
reward boundary, reward:  -5021.539483073175
              dist_rew :  -321.53948307317376
reward boundary, reward:  -2177.2414462663814
              dist_rew :  832.758553733619
reward boundary, reward:  -5194.276347533745
              dist_rew :  -604.2763475337466
reward